문제 정의 : SVM(Support vector machine) 사용하여 타이타닉 생존자(1), 사망(0) 예측

기본 라이브러리 불러오기

In [1]:
import pandas as pd
import seaborn as sns

한글 깨짐 방지

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()


fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


[Step 1] 데이터 준비 - seaborn에서 제공하는 titanic 데이터셋 가져오기

In [3]:
# loda_dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset('titanic')

[Step 2] 데이터 탐색 및 데이터 전처리

In [4]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [6]:
# NaN값이 많은 deck(객실 데크 위치) 열을 삭제, embarked와 내용이 겹치는 embark_town(승선 도시) 열을 삭제

rdf = df.drop(['deck','embark_town'], axis=1) # 열을 삭제할거라 axis=1
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    891 non-null    int64   
 1   pclass      891 non-null    int64   
 2   sex         891 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       891 non-null    int64   
 5   parch       891 non-null    int64   
 6   fare        891 non-null    float64 
 7   embarked    889 non-null    object  
 8   class       891 non-null    category
 9   who         891 non-null    object  
 10  adult_male  891 non-null    bool    
 11  alive       891 non-null    object  
 12  alone       891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 72.4+ KB


In [7]:
# age 열에 나이 데이터가 없는 모든 행을 삭제 -> 177개 NaN 삭제
rdf = rdf.dropna(subset=['age'],how='any', axis=0) # age 열을 기준으로 모든 데이터 삭제
                                                   # axis=0 다 행이기 때문에
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    712 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [9]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값 확인 -> 탑승한 도시의 첫글자
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
most_freq

'S'

In [10]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
rdf['embarked'].fillna(most_freq, inplace=True)

[Step 3] 분석에 사용할 속성을 선택

In [11]:
# 분석에 활용할 열(속성)을 선택 (생존 여부, 객실 등급, 성별, 나이, 타이타닉에 탑승한 형제 자매수,
# 타이타닉에 탑승한 부모/자녀 수, 탑승한 곳(항구), C=Cherbourg, Q=Queenstown, S=Southhampton
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [12]:
# 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변경
# male, female -> [1,0],[0,1]
# embarked c, Q, s -> [1,0,0],[0,1,0],[0,0,1]

onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf,onehot_sex], axis=1)     # 데이터프레임 2개 합침

onehot_embarked = pd.get_dummies(ndf['embarked'],prefix='town') # 그냥 s,q,c 만있으면 몰라서 
ndf = pd.concat([ndf,onehot_embarked], axis=1)     # 데이터프레임 2개 합침

ndf

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,Q,1,0,0,1,0
886,0,2,male,27.0,0,0,S,0,1,0,0,1
887,1,1,female,19.0,0,0,S,1,0,0,0,1
889,1,1,male,26.0,0,0,C,0,1,1,0,0


In [13]:
# 기존 칼럼 삭제
ndf.drop(['sex','embarked'],axis=1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


[Step 4] 데이터셋 구분 - 훈련용(train data)/검증용(test data)

In [14]:
# 속성(변수) 선택
# ndf -> X, y 형태로 분리하는 작업 (문제집, 정답지)
X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S']]
y = ndf['survived']


In [27]:
# 설명 변수(데이터, 문제집) 정규화 0~100까지 연령값이 많아서 -> 스케일링 0~1사이의 작은 값으로 축소
# 모든 변수들 각 column 별로 0~1 사이로 스케일링
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)
X

array([[ 0.91123237, -0.53037664,  0.52457013, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364,  0.57183099,  0.52457013, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237, -0.25482473, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       ...,
       [-1.47636364, -0.73704057, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364, -0.25482473, -0.55170307, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237,  0.15850313, -0.55170307, ..., -0.47180795,
         4.94974747, -1.87589641]])

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)

(499, 9)
(215, 9)


[Step 5] SVM 분류 모형 - sklearn 사용

In [29]:
# sklearn 라이브러리에서 svm 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
# 벡터 공간을 맵핑 하는 함수 -> radial basis function (RBF) 커널을 사용
# 선형(linear), 다항식(poly), 가우시안 RBF(rbf) , 시그모이드(sigmoid)

svm_model = svm.SVC(kernel='rbf') # 공간 개념인 벡터를 kernel이라고 함 그곳에 넣는것

In [30]:
# 훈련 데이터를 사용하여 모형 학습
svm_model.fit(X_train, y_train)

# 테스트 데이터를 사용하여 y_hat 예측
y_hat = svm_model.predict(X_test)

[Step 6] 모형 성능 평가

In [31]:
from sklearn import metrics

print("테스트 세트 점수 : {:.2f}%".format(svm_model.score(X_test, y_test)*100))

테스트 세트 점수 : 80.93%


In [32]:
# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, y_hat)
print(svm_report)

              precision    recall  f1-score   support

           0       0.79      0.91      0.85       126
           1       0.84      0.66      0.74        89

    accuracy                           0.81       215
   macro avg       0.82      0.79      0.80       215
weighted avg       0.81      0.81      0.80       215

